### 04 - kNN classification
De dataset met alle keypoints en aanvullende informatie zal gebruikt worden om een kNN classificatiemodel te trainen.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import pickle

#### Stap 1. Train- en testdata genereren

In [17]:
# Keuze voor model maken

train_df = pd.read_csv('keypoints_train_df_YOLOv8m.csv', index_col=0)
test_df = pd.read_csv('keypoints_test_df_YOLOv8m.csv', index_col=0)

#train_df = pd.read_csv('keypoints_train_df_YOLOv8n.csv', index_col=0)
#test_df = pd.read_csv('keypoints_test_df_YOLOv8n.csv', index_col=0)

In [18]:
train_df.head()

,path,pose,pose_code,confidence,x00,y00,x01,y01,x02,y02,...,v14,v15,v16,all fours,ver dist hips,standing,hor_dist_ankles,hor dist wrists,ver dist wrists,ver dist ankles
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.934024,0.423500,0.709093,0.402510,0.702865,0.427802,0.704759,...,0.862971,0.845818,0.790219,0.084437,-0.477136,0.083704,0.008652,0.602298,-0.498172,0.008652
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.932599,0.336725,0.687173,0.331049,0.671780,0.337968,0.669529,...,0.940688,0.942040,0.897956,0.077501,-0.418508,0.184631,0.042990,0.003060,-0.673096,0.042990
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.936376,0.379543,0.704821,0.363566,0.690626,0.378991,0.699057,...,0.924674,0.896926,0.840897,0.102676,-0.399451,0.113263,0.030958,0.243853,-0.503882,0.030958
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.933815,0.616935,0.697705,0.620064,0.689498,0.625734,0.677587,...,0.967766,0.868041,0.914164,0.037754,-0.471743,0.237028,0.025091,0.000562,-0.672612,0.025091
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.898250,0.454746,0.786763,0.441597,0.773290,0.450748,0.781609,...,0.816905,0.630279,0.528475,0.088247,-0.199260,0.047326,0.024270,0.103216,-0.265029,0.024270


In [19]:
X_train = train_df.iloc[:, 4:62]
X_train

,x00,y00,x01,y01,x02,y02,x03,y03,x04,y04,...,v14,v15,v16,all fours,ver dist hips,standing,hor_dist_ankles,hor dist wrists,ver dist wrists,ver dist ankles
0,0.423500,0.709093,0.402510,0.702865,0.427802,0.704759,0.382130,0.664700,0.443019,0.669867,...,0.862971,0.845818,0.790219,0.084437,-0.477136,0.083704,0.008652,0.602298,-0.498172,0.008652
1,0.336725,0.687173,0.331049,0.671780,0.337968,0.669529,0.343589,0.613480,0.347272,0.606030,...,0.940688,0.942040,0.897956,0.077501,-0.418508,0.184631,0.042990,0.003060,-0.673096,0.042990
2,0.379543,0.704821,0.363566,0.690626,0.378991,0.699057,0.356653,0.627904,0.382242,0.644900,...,0.924674,0.896926,0.840897,0.102676,-0.399451,0.113263,0.030958,0.243853,-0.503882,0.030958
3,0.616935,0.697705,0.620064,0.689498,0.625734,0.677587,0.638563,0.643493,0.633839,0.619321,...,0.967766,0.868041,0.914164,0.037754,-0.471743,0.237028,0.025091,0.000562,-0.672612,0.025091
4,0.454746,0.786763,0.441597,0.773290,0.450748,0.781609,0.420292,0.747943,0.444179,0.770287,...,0.816905,0.630279,0.528475,0.088247,-0.199260,0.047326,0.024270,0.103216,-0.265029,0.024270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,0.421135,0.791333,0.408298,0.804570,0.453501,0.796470,0.386294,0.788566,0.506973,0.767220,...,0.344972,0.261209,0.238041,0.148163,-0.422612,-0.329379,0.017819,0.828508,0.118531,0.017819
1992,0.153706,0.166005,0.172928,0.123942,0.120859,0.155429,0.243908,0.132283,0.109884,0.211759,...,0.959405,0.913678,0.891697,0.085577,0.444358,0.719394,0.041055,0.659261,-0.487285,0.041055
1993,0.546612,0.345693,0.560153,0.347666,0.542893,0.332598,0.565480,0.393461,0.522904,0.356668,...,0.959685,0.873797,0.812785,0.188189,0.313564,0.574979,0.349487,0.064427,0.019060,0.349487
1994,0.449509,0.451181,0.451348,0.424801,0.446124,0.431690,0.472965,0.387391,0.465405,0.400063,...,0.987193,0.960378,0.911958,0.014271,0.268840,0.413430,0.013747,0.010388,-0.636566,0.013747


In [20]:
y_train = train_df['pose_code']
y_train

0       0
1       0
2       0
3       0
4       0
       ..
1991    5
1992    5
1993    5
1994    5
1995    5
Name: pose_code, Length: 1996, dtype: int64

In [21]:
X_test = test_df.iloc[:, 4:62]
X_test

,x00,y00,x01,y01,x02,y02,x03,y03,x04,y04,...,v14,v15,v16,all fours,ver dist hips,standing,hor_dist_ankles,hor dist wrists,ver dist wrists,ver dist ankles
0,0.513715,0.741517,0.497201,0.735360,0.509464,0.740671,0.476493,0.706399,0.494905,0.724907,...,0.872600,0.846585,0.785794,0.105069,-0.477905,0.111305,0.012622,0.361397,-0.571468,0.012622
1,0.580441,0.789922,0.580974,0.787706,0.593028,0.778452,0.587604,0.751723,0.604622,0.734322,...,0.956924,0.804400,0.881734,0.067210,-0.441707,0.073366,0.019571,0.368055,-0.341899,0.019571
2,0.646324,0.715999,0.652871,0.698378,0.652485,0.699054,0.671895,0.655805,0.646459,0.660106,...,0.981466,0.954894,0.949070,0.061583,-0.370940,0.177699,0.081472,0.020100,-0.582963,0.081472
3,0.559559,0.754270,0.560810,0.757166,0.579860,0.739475,0.572747,0.750394,0.606282,0.711236,...,0.955897,0.837674,0.884011,0.062984,-0.446968,0.092755,0.016372,0.527511,-0.400786,0.016372
4,0.349407,0.688239,0.341974,0.666379,0.346537,0.666175,0.328168,0.633742,0.330032,0.634200,...,0.948830,0.951415,0.931818,0.143123,-0.335726,0.080581,0.016003,0.037134,-0.568170,0.016003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.420442,0.363706,0.428557,0.353078,0.421531,0.348491,0.456424,0.361001,0.434070,0.354083,...,0.956419,0.916596,0.868035,0.450943,0.282212,0.531001,0.351565,0.375512,-0.007998,0.351565
453,0.563601,0.240097,0.569275,0.214819,0.541831,0.221728,0.557320,0.220631,0.483589,0.233351,...,0.989875,0.887248,0.926128,0.554387,0.396655,0.664607,0.664052,0.791373,0.021863,0.664052
454,0.443184,0.195190,0.473011,0.173855,0.427609,0.172129,0.539675,0.190345,0.412880,0.186139,...,0.985922,0.941767,0.912613,0.607049,0.389717,0.696443,0.456063,0.692327,0.072614,0.456063
455,0.328837,0.033087,0.339197,0.000000,0.331087,0.008918,0.408413,0.003092,0.365714,0.026697,...,0.611221,0.311374,0.229609,0.762898,0.611900,1.000000,0.699720,0.922251,-0.057706,0.699720


In [22]:
y_test = test_df['pose_code']
y_test

0      0
1      0
2      0
3      0
4      0
      ..
452    4
453    4
454    4
455    4
456    4
Name: pose_code, Length: 457, dtype: int64

#### Stap 2. Model trainen

TO DO: GridSearchCV, spelen met features, kijken welk model het beter doet
TO DO: o.b.v. eigen logica

In [23]:
knn = KNeighborsClassifier()
knn.set_params(n_neighbors = 5, weights = 'distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(weights='distance')

#### Stap 3. Model evalueren


In [24]:
y_train_pred = knn.predict(X_train)
accuracy_score(y_train, y_train_pred)

0.998496993987976

In [25]:
confusion_matrix(y_train, y_train_pred)

array([[375,   0,   0,   0,   0,   0],
       [  0, 305,   0,   0,   1,   0],
       [  0,   0, 498,   0,   0,   0],
       [  0,   0,   0, 299,   0,   0],
       [  0,   0,   0,   0, 472,   0],
       [  0,   1,   1,   0,   0,  44]], dtype=int64)

In [26]:
y_test_pred = knn.predict(X_test)
accuracy_score(y_test, y_test_pred)

0.9824945295404814

In [27]:
knn.predict_proba(X_test)

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [28]:
confusion_matrix(y_test, y_test_pred)

array([[ 93,   0,   0,   0,   0],
       [  0,  72,   1,   2,   2],
       [  0,   0, 112,   1,   0],
       [  0,   0,   0,  69,   0],
       [  0,   2,   0,   0, 103]], dtype=int64)

#### Stap 4. Model en voorspellingen exporteren

In [29]:
# Prediction en proba toevoegen aan DataFrame

train_df['prediction'] = y_train_pred
train_df['proba'] = knn.predict_proba(X_train).max(axis=1)
train_df.head()

,path,pose,pose_code,confidence,x00,y00,x01,y01,x02,y02,...,v16,all fours,ver dist hips,standing,hor_dist_ankles,hor dist wrists,ver dist wrists,ver dist ankles,prediction,proba
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.934024,0.423500,0.709093,0.402510,0.702865,0.427802,0.704759,...,0.790219,0.084437,-0.477136,0.083704,0.008652,0.602298,-0.498172,0.008652,0,1.0
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.932599,0.336725,0.687173,0.331049,0.671780,0.337968,0.669529,...,0.897956,0.077501,-0.418508,0.184631,0.042990,0.003060,-0.673096,0.042990,0,1.0
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.936376,0.379543,0.704821,0.363566,0.690626,0.378991,0.699057,...,0.840897,0.102676,-0.399451,0.113263,0.030958,0.243853,-0.503882,0.030958,0,1.0
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.933815,0.616935,0.697705,0.620064,0.689498,0.625734,0.677587,...,0.914164,0.037754,-0.471743,0.237028,0.025091,0.000562,-0.672612,0.025091,0,1.0
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,downdog,0,0.898250,0.454746,0.786763,0.441597,0.773290,0.450748,0.781609,...,0.528475,0.088247,-0.199260,0.047326,0.024270,0.103216,-0.265029,0.024270,0,1.0


In [33]:
# DataFrame exporteren
# Keuze maken voor model

train_df.to_csv('train_df_YOLOv8m.csv')
#train_df.to_csv('train_df_YOLOv8n.csv')

In [34]:
# Pickle maken
# Keuze maken voor model

pickle.dump(knn, open('knn_YOLOv8m.pkl', 'wb'))
#pickle.dump(knn, open('knn_YOLOv8n.pkl', 'wb'))